In [1]:
import sys

sys.path.append("../")

import os

import hydra
import numpy as np
import torch
import torch.nn.functional as F
import wandb
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig
from termcolor import cprint
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from tqdm import tqdm

from lib.datasets import ThingsMEGDataset
from lib.models import BasicConvClassifier
from lib.utils import set_seed

data_dir = "../../data"
batch_size = 32
num_workers = 1
loader_args = {"batch_size": batch_size, "num_workers": num_workers}


In [2]:
from lib.datasets import ThingsMEGDatasetWithImages
train_set_with_images = ThingsMEGDatasetWithImages("train", data_dir)
train_loader_with_images = DataLoader(train_set_with_images, shuffle=False, **loader_args)

In [8]:
print(train_set_with_images.categories.index("camera1"))
print(train_set_with_images.categories.index("camera2"))

246
247


In [3]:
for image_X, brain_X, y, subject_idx in train_loader_with_images:
    # print(image_X.shape)
    # print(brain_X.shape)
    # print(y.shape)
    # print(subject_idx.shape)
    break

    

198
ice_pack ice-cream_cone
262
crystal1 crystal_ball
401
hot-air_balloon hot_chocolate
404
ice_cream ice_cube
448
camera1 camera_lens
766
ice-cream_cone ice_cream
835
camera2 camera1
840
pepper2 pepper1
924
crystal2 crystal1
948
pepper_mill pepper2
1081
chicken2 chicken1
1131
chicken1 chicken_wire
1182
ice_cube ice_pack
1192
hot_tub hot-water_bottle
1201
pepper1 pepper_mill
1244
hot-water_bottle hot_tub
1332
hot_chocolate hot-air_balloon
1562
camera_lens camera2
1911
crystal_ball crystal2
1981
chicken_wire chicken2
2186
crystal2 crystal1
2219
pepper2 pepper1
2234
camera_lens camera2
2306
hot_chocolate hot-air_balloon
2442
hot_tub hot-water_bottle
2481
chicken1 chicken_wire
2524
chicken2 chicken1
2798
ice_pack ice-cream_cone
2867
crystal_ball crystal2
2911
pepper1 pepper_mill
3012
ice-cream_cone ice_cream
3076
hot-water_bottle hot_tub
3304
hot-air_balloon hot_chocolate
3403
ice_cream ice_cube
3566
camera1 camera_lens
3755
chicken_wire chicken2
3871
pepper_mill pepper2
3891
crystal1 cry

In [5]:
print(train_set_with_images.error_count)

0


In [ ]:
print()